<a href="https://colab.research.google.com/github/BahodirML/Coding_Practices/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-cloud-bigquery google-cloud-bigquery-storage

In [ ]:
from google.colab import auth
auth.authenticate_user()

# Setup
- install
- authenticatiopn

# Load cronos.labs campain data

In [ ]:
import gdown
import zipfile
import io
import os
import pandas as pd
url = 'https://drive.google.com/uc?id=1MGvcUsln5vykGQj3tkpUds3RtwotQNb7'
output = 'cronos_enriched.zip'

if not os.path.exists(output):
    # Download the file if it does not exist
    gdown.download(url, output, quiet=False)

with zipfile.ZipFile(output) as z:
    with z.open(z.namelist()[0]) as f:
        enriched_df = pd.read_csv(io.BytesIO(f.read()))

enriched_df.head()

Downloading...
From (original): https://drive.google.com/uc?id=1MGvcUsln5vykGQj3tkpUds3RtwotQNb7
From (redirected): https://drive.google.com/uc?id=1MGvcUsln5vykGQj3tkpUds3RtwotQNb7&confirm=t&uuid=0d65a672-95fc-466a-8408-03cfb66a097f
To: /content/cronos_enriched.zip
100%|██████████| 155M/155M [00:02<00:00, 72.7MB/s]
<ipython-input-5-affeeac3b7a1>:15: DtypeWarning: Columns (11,52,53,177,329,335,369,370,372,413,422,430,511) have mixed types. Specify dtype option on import or set low_memory=False.
  enriched_df = pd.read_csv(io.BytesIO(f.read()))


,DFP_js_intlDT_intlDateTimeFormat,DFP_js_intlDT_hourCycle,DFP_js_intlDT_locale,DFP_js_intlDT_executionTime,DFP_js_intlDT_timeZone,DFP_js_intlDT_numberingSystem,DFP_js_intlDT_calendar,DFP_js_viewport_client,DFP_js_viewport_executionTime,DFP_js_viewport_inner,...,Lie_heuristic_weight_score,Lie_rule_flag,Lie_rule_flag_count,bot_category_heuristics,CascM_mouseMovement_botness,ML_botness_probability,H_botness_probability,Lie_botness_probability,botness_score,combined_bot_category
0,"June 3, 2024 at 7:39:15 PM Central European Su...",h12,en-US,0.4,Europe/Rome,latn,gregory,1530x848,0.2,1545x848,...,0.4,True,2,Human,0.924927,0.749629,0.181818,1.0,0.990000,Bot
1,"June 3, 2024 at 9:05:45 PM Iran Standard Time",h12,en-US,1.2,Asia/Tehran,latn,gregory,407x798,0.9,407x798,...,0.0,False,0,Human,NaN,0.015019,0.090909,0.0,0.052964,Human
2,"June 1, 2024 at 12:10:14 PM West Kazakhstan Time",h12,en-US,0.2,Asia/Qyzylorda,latn,gregory,1519x730,0.1,1536x730,...,0.0,False,0,Likely Bot,0.800494,0.218462,0.272727,0.0,0.391103,Suspicious
3,"June 1, 2024 at 1:13:00 AM Uzbekistan Standard...",h12,en-US,0.5,Asia/Tashkent,latn,gregory,1680x881,0.2,1680x881,...,0.0,False,0,Likely Bot,0.776429,0.419831,0.318182,0.0,0.458156,Likely Bot
4,"June 1, 2024 at 12:58:27 AM Uzbekistan Standar...",h12,en-US,0.4,Asia/Tashkent,latn,gregory,1200x750,0.2,1200x750,...,0.4,True,8,Bot,0.989639,0.997214,0.454545,1.0,0.990000,Bot


# New Section

In [ ]:
enriched_df['ID_social_id'].value_counts()

ID_social_id
0x90fb225dd07dead01d3322ff566cc40668fa1244    20
0x652b2031eb06be326ab2abb44da91c1324d5ecdb    12
0x16d5b231dc27ed5a2b0b80ec4e188314c49b1698    11
0x449c4291fc18e8c1acf3a2902cf06082723a3303     9
0xe9d94fdd28c32599ee1757a89c078eb375d8f52d     8
                                              ..
0xbb331f7c9c6508ad76853fa5926673839c149951     1
0x5a8984f700b6d83086fbd576e513a08822329438     1
0x6f533803064962a96f7fd7457f94309106edf934     1
0xe611038f2ef1cda4b7c8b5bf74174b3f4b67bf46     1
0x525c49bf83ce3a1aaf425ac1a463537db68c8bd7     1
Name: count, Length: 5994, dtype: int64

# Download BQ data for wallets of interest

In [ ]:
from google.cloud import bigquery
from google.cloud import bigquery_storage_v1
# Specify the project ID, dataset ID, and table ID
project_id = "innerworks-qa"
dataset_id = "public_preview___blockchain_analytics_cronos_mainnet"
table_id = "transactions"
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

# Initialize the BigQuery client
#client = bigquery.Client()
client = bigquery.Client(project=project_id)

bqstorageclient = bigquery_storage_v1.BigQueryReadClient()



# Get the dataset information
try:
    # Make an API request.
    dataset = client.get_dataset(f"{project_id}.{dataset_id}")
    dataset_location = dataset.location
    print(f"Dataset {project_id}.{dataset_id} is in the {dataset_location} region.")
except Exception as e:
    print(f"Error retrieving dataset details: {e}")
    dataset_location = "US"  # Defaulting to US if not found

Error retrieving dataset details: 403 GET https://bigquery.googleapis.com/bigquery/v2/projects/innerworks-qa/datasets/public_preview___blockchain_analytics_cronos_mainnet?prettyPrint=false: Access Denied: Dataset innerworks-qa:public_preview___blockchain_analytics_cronos_mainnet: Permission bigquery.datasets.get denied on dataset innerworks-qa:public_preview___blockchain_analytics_cronos_mainnet (or it may not exist).


In [ ]:
# Import necessary libraries
from tqdm import tqdm
import pandas as pd

# Get all unique cronos addresses
cronos_addresses = enriched_df['ID_social_id'].unique().tolist()

# Convert the list to a comma-separated string of quoted addresses
cronos_addresses_str = ','.join(
    [f"'{address}'" for address in cronos_addresses])

# Define the query
query_string = f"""
SELECT *
FROM `{full_table_id}`
WHERE from_address IN ({cronos_addresses_str}) OR to_address IN ({cronos_addresses_str})
LIMIT 10000000
"""

# Initialize variables
error_count = 0

# Run the query and convert the results to a DataFrame
try:
    # Set the correct location
    query_job = client.query(query_string, location=dataset_location)
    wallet_df = query_job.result().to_dataframe(bqstorage_client=bqstorageclient)
except Exception as e:
    error_count += 1
    print(f"Error (count {error_count}) querying the dataset: {e}")

wallet_df

Error (count 1) querying the dataset: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/innerworks-qa/jobs?prettyPrint=false: Access Denied: Project innerworks-qa: User does not have bigquery.jobs.create permission in project innerworks-qa.

Location: US
Job ID: f3a0bb52-3ea5-4ade-a824-2e7b8b967456



NameError: name 'wallet_df' is not defined

# Save wallet Info

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
wallet_df.to_csv('/content/drive/MyDrive/HBAI/projects/Innerworks: botnet detection/v2Report/endOfCampaignWalletData.csv')

# Load wallet Info

In [ ]:
# Install gdown if not already installed
!pip install gdown

# Import necessary libraries
import gdown
import pandas as pd

# File ID from the shared link
file_id = "1V3aT7QqwfGVAeHPWBPk9w6bkAS_FK22d"

# URL for gdown to download the file
url = f"https://drive.google.com/uc?id={file_id}"

# Output file path
output = "endOfCampaignWalletData.csv"

# Download the file
gdown.download(url, output, quiet=False)

# Load the CSV file into a pandas DataFrame
wallet_df = pd.read_csv(output)

# Display the DataFrame
wallet_df.head()

Downloading...
From (original): https://drive.google.com/uc?id=1V3aT7QqwfGVAeHPWBPk9w6bkAS_FK22d
From (redirected): https://drive.google.com/uc?id=1V3aT7QqwfGVAeHPWBPk9w6bkAS_FK22d&confirm=t&uuid=cb746090-47b5-46ca-a0c4-cf794c2358f5
To: /content/endOfCampaignWalletData.csv
100%|██████████| 2.33G/2.33G [00:29<00:00, 78.6MB/s]


,Unnamed: 0,block_hash,block_timestamp,transaction_hash,transaction_index,nonce,from_address,to_address,value,input,gas,gas_price,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,access_list,r,s,v
0,0,0xcfb7781c4f09639d33ef9ddbc8cffcb6684329bb6caa...,2024-04-11 08:14:36+00:00,0xd3bd569eebf38c0c4b53002c5801e40d65089061091a...,0,1882.0,0x306f43639ee3172e16798731029055dd2691499b,0x14fb0e7640e7fb7765cfa87cec973ff5465d1c66,"{'string_value': '0', 'bignumeric_value': Deci...",0x095ea7b30000000000000000000000004e0186e2cd19...,51657,"{'string_value': '5162740480769', 'bignumeric_...",5.162740e+12,1.627405e+11,2,[],{'string_value': '4473517533020926627782634415...,{'string_value': '4790959909023628722489207200...,"{'string_value': '1', 'bignumeric_value': None}"
1,1,0x0692283a8e0dc343ec0f2758c99a9195800b8132e053...,2024-04-30 13:00:55+00:00,0x2771d36dbcb97bd2638badc75762d6c36115bf5a0489...,8,5786.0,0x26de9cdca752ddb91ae78c7ef895902d7747fd71,0xce3f4e59834b5b52b301e075c5b3d427b6884b3d,"{'string_value': '0', 'bignumeric_value': Deci...",0xa22cb4650000000000000000000000007a3cdb2364f9...,34131,"{'string_value': '5036652423771', 'bignumeric_...",6.786652e+12,3.665242e+10,2,[],{'string_value': '8306547203907244710235670689...,{'string_value': '1232090997641214761327500931...,"{'string_value': '1', 'bignumeric_value': None}"
2,2,0x59b0a002edc60398ab6b20b1cf8708f5452cb8450bba...,2024-04-30 04:50:00+00:00,0x8a489630160e29798f7acbe3593880bb17c9fff5a145...,18,9.0,0x95a432c3af672b714d7d5fe90b196297cb53549b,0x55b5b0562fb3da3b4108145cd30e6d3b6e16c69c,"{'string_value': '1000000000000000000', 'bignu...",0x00d5a9e8000000000000000000000000000000000000...,247501,"{'string_value': '5045590000000', 'bignumeric_...",6.795590e+12,4.559000e+10,2,[],{'string_value': '3141878500829044557533286526...,{'string_value': '2303971006535084978768371022...,"{'string_value': '1', 'bignumeric_value': None}"
3,3,0x8191f729a94a5d0b477585b022e4e165e988c0ba7af7...,2024-02-14 08:57:58+00:00,0x584dc50bce0571f6a4e92729ab423401ccb20617ee9a...,4,2838.0,0x62cd39160bdefbbb149166bc667a98a4f9b141da,0xd988909fc611a9d8eb4141bf37b98f815a8820ce,"{'string_value': '0', 'bignumeric_value': Deci...",0x646174613a2c7b2270223a226372632d3230222c226f...,21992,"{'string_value': '10629435893970', 'bignumeric...",1.062944e+13,6.294359e+11,2,[],{'string_value': '8239761473912596102512572072...,{'string_value': '5440176852016776338204304754...,"{'string_value': '1', 'bignumeric_value': None}"
4,4,0xff25686090ee04a7c02264e190228788d766892bf697...,2024-06-04 14:32:17+00:00,0x067b7e50f6ffb11b1a4629b6e8a3ed178d1fda92eaf0...,6,11717.0,0x10337523fe988d6ce28b0bd65c4cc6dd34e39c1d,0x11703dc41b6417e9e4b7f560710ef2ef5e9eb1ca,"{'string_value': '0', 'bignumeric_value': Deci...",0x9b6522d5000000000000000000000000bf4e430cd0ce...,301140,"{'string_value': '5045633650000', 'bignumeric_...",6.795634e+12,4.563365e+10,2,[],{'string_value': '1113539345344722326541772922...,{'string_value': '5199957291835320997655667863...,"{'string_value': '1', 'bignumeric_value': None}"


In [ ]:
wallet_df.block_timestamp.sort_values()

379950    2021-11-08 07:35:29+00:00
23607     2021-11-08 07:46:22+00:00
269855    2021-11-08 07:47:34+00:00
235398    2021-11-08 08:12:31+00:00
316036    2021-11-08 08:15:02+00:00
                    ...            
430884    2024-06-07 12:45:45+00:00
415239    2024-06-07 12:45:56+00:00
8240      2024-06-07 12:46:13+00:00
336620    2024-06-07 12:46:13+00:00
277500    2024-06-07 12:46:24+00:00
Name: block_timestamp, Length: 1818504, dtype: object

In [ ]:

# Check if these unique to_address values are in the ID_social_id column of enriched_df
is_in_enriched_df = pd.Series(wallet_df['to_address'].unique()).isin(enriched_df['ID_social_id'].unique())

# Sum the boolean values to get the count
count = is_in_enriched_df.sum()

# Print the result
print(f"{count}/{wallet_df['to_address'].nunique()}")

4668/18988


In [ ]:

# Check if these unique to_address values are in the ID_social_id column of enriched_df
is_in_enriched_df = pd.Series(wallet_df['from_address'].unique()).isin(enriched_df['ID_social_id'].unique())

# Sum the boolean values to get the count
count = is_in_enriched_df.sum()

# Print the result
print(f"{count}/{wallet_df['from_address'].nunique()}")

4715/8142


In [ ]:
wallet_df['from_address'].nunique()

In [ ]:
df = wallet_df

# Bot detection

In [ ]:
import pandas as pd

# Assuming df is the DataFrame with transaction data
# Load your data into a DataFrame
# Replace 'your_data.csv' with the path to your CSV file
# df = pd.read_csv('your_data.csv')

# Define functions for each rule
def detect_high_frequency_transactions(df, threshold):
    frequent_transactions = df.groupby('from_address').size()
    suspicious_addresses = frequent_transactions[frequent_transactions > threshold].index
    return suspicious_addresses

def detect_small_repetitive_transactions(df, max_value):
    df['value_bignumeric'] = df['value'].apply(lambda x: x['bignumeric_value'])
    small_transactions = df[df['value_bignumeric'] <= max_value]
    repetitive_transactions = small_transactions.groupby('from_address').size()
    suspicious_addresses = repetitive_transactions[repetitive_transactions > 5].index  # More stringent
    return suspicious_addresses

def detect_multiple_wallets_same_addresses(df, min_interactions):
    interactions = df.groupby('to_address')['from_address'].nunique()
    suspicious_addresses = interactions[interactions >= min_interactions].index
    return suspicious_addresses

def detect_new_wallets_high_activity(df, recent_days, min_transactions):
    df['block_timestamp'] = pd.to_datetime(df['block_timestamp'])
    if df['block_timestamp'].dt.tz is None:
        df['block_timestamp'] = df['block_timestamp'].dt.tz_localize('UTC')
    recent_wallets = df[df['block_timestamp'] >= (pd.Timestamp.now(tz='UTC') - pd.Timedelta(days=recent_days))]
    high_activity_wallets = recent_wallets.groupby('from_address').size()
    suspicious_addresses = high_activity_wallets[high_activity_wallets >= min_transactions].index
    return suspicious_addresses

def detect_repeated_gas_usage(df, min_repeats):
    repeated_gas = df.groupby('from_address')['gas'].apply(lambda x: x.value_counts().max())
    suspicious_addresses = repeated_gas[repeated_gas >= min_repeats].index
    return suspicious_addresses

# Combine rules to detect suspicious wallets
def detect_suspicious_wallets(df):
    # More stringent parameters for higher precision
    threshold = 100  # Higher threshold for high frequency transactions
    max_value = 1e17  # Lower maximum value for small transactions
    recent_days = 15  # Fewer recent days for new wallets
    min_interactions = 3  # Minimum unique interactions for multiple wallets
    min_transactions = 10  # Minimum transactions for high activity new wallets
    min_repeats = 3  # Minimum repeats for gas usage

    high_freq_suspicious = detect_high_frequency_transactions(df, threshold)
    small_repetitive_suspicious = detect_small_repetitive_transactions(df, max_value)
    multiple_wallets_suspicious = detect_multiple_wallets_same_addresses(df, min_interactions)
    new_wallets_suspicious = detect_new_wallets_high_activity(df, recent_days, min_transactions)
    repeated_gas_suspicious = detect_repeated_gas_usage(df, min_repeats)

    # Combine all suspicious addresses
    suspicious_addresses = set(high_freq_suspicious).intersection(
        small_repetitive_suspicious,
        multiple_wallets_suspicious,
        new_wallets_suspicious,
        repeated_gas_suspicious
    )

    return suspicious_addresses

# Example usage
# df = pd.read_csv('your_data.csv')  # Load your data here
suspicious_wallets = detect_suspicious_wallets(enriched_df)
print(suspicious_wallets)


KeyError: 'from_address'

In [ ]:
df['value_bignumeric'].value_counts().sort_index()

NameError: name 'df' is not defined

In [ ]:
df['nonce'].astype('int').plot(kind='hist')

In [ ]:
_M = df['nonce']>
df['nonce'].astype('int').plot(kind='hist')

In [ ]:
_M = df['value_bignumeric']<
df['value_bignumeric'].astype('float').plot(kind='hist')

In [ ]:
df.shape, len(suspicious_wallets)

In [ ]:
len(set(suspicious_wallets))

In [ ]:
enriched_df['is_suspicious'] = enriched_df['ID_social_id'].isin(suspicious_wallets)


In [ ]:
_M = enriched_df['is_suspicious']
enriched_df[_M]['total_raw_scores'].plot(kind='hist', color='green')
enriched_df[~_M]['total_raw_scores'].plot(kind='hist', color='blue')


KeyError: 'is_suspicious'

In [ ]:
!pip install plotly


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Assuming _M is already defined as enriched_df['is_suspicious']
_M = enriched_df['is_suspicious']

# Create histograms for suspicious and non-suspicious wallets
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=enriched_df[_M]['total_raw_scores'],
    nbinsx=30,
    name='Suspicious Wallets',
    marker_color='green',
    opacity=0.75
))

fig.add_trace(go.Histogram(
    x=enriched_df[~_M]['total_raw_scores'],
    nbinsx=30,
    name='Non-Suspicious Wallets',
    marker_color='blue',
    opacity=0.75
))

# Update layout
fig.update_layout(
    title='Distribution of Total Raw Scores',
    xaxis_title='Total Raw Scores',
    yaxis_title='Count',
    barmode='overlay',
    template='plotly_white'
)

fig.update_traces(opacity=0.6)

# Show plot
fig.show()


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Assuming _M is already defined as enriched_df['is_suspicious']
_M = enriched_df['is_suspicious']

# Create histograms for suspicious and non-suspicious wallets
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=enriched_df[_M]['Lie_rule_flag'],
    nbinsx=30,
    name='Suspicious Wallets',
    marker_color='green',
    opacity=0.75
))

fig.add_trace(go.Histogram(
    x=enriched_df[~_M]['Lie_rule_flag'],
    nbinsx=30,
    name='Non-Suspicious Wallets',
    marker_color='blue',
    opacity=0.75
))

# Update layout
fig.update_layout(
    title='Distribution of Lie_rule_flag',
    xaxis_title='Total Raw Scores',
    yaxis_title='Count',
    barmode='overlay',
    template='plotly_white'
)

fig.update_traces(opacity=0.6)

# Show plot
fig.show()
